### Email exploration. 

<br>

#### General Setup.
___

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from IPython import display
from ipywidgets import interact, widgets

%matplotlib inline

In [2]:
plt.style.use('seaborn-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)

<br>

#### Loading the data.
___